In [1]:
!pip install pyshark

## Network Traffic Analyzer using Python and PyShark
## Developed for monitoring and analyzing network traffic using CCNAv7 concepts (VLANs, routing)
### Tech Stack: Python, NumPy, Pandas, PyShark


In [2]:
import pandas as pd
import numpy as np
import pyshark
import asyncio

In [ ]:

class NetworkTrafficAnalyzer:
    def __init__(self, interface):
        self.interface = interface
        self.capture = None
        self.data = []

    def start_capture(self):
        print(f"Starting capture on interface: {self.interface}")
        self.capture = pyshark.LiveCapture(interface=self.interface)

    def analyze_packet(self, packet):
        try:
            protocol = packet.transport_layer if packet.transport_layer else 'Unknown'
            src_ip = packet.ip.src if hasattr(packet, 'ip') else 'N/A'
            dst_ip = packet.ip.dst if hasattr(packet, 'ip') else 'N/A'
            length = int(packet.length) if hasattr(packet, 'length') else 0
            vlan_id = packet.vlan.id if hasattr(packet, 'vlan') else 'N/A'
            return {'Protocol': protocol, 'Source IP': src_ip, 'Destination IP': dst_ip, 'Length': length, 'VLAN ID': vlan_id}
        except AttributeError as e:
            print(f"Skipping packet due to missing attributes: {e}")
            return None

    async def analyze_traffic_async(self, packet_count=50):
        print("Analyzing traffic...")
        try:
            async for packet in self.capture.sniff_continuously(packet_count=packet_count):
                analysis = self.analyze_packet(packet)
                if analysis:
                    self.data.append(analysis)
            if self.data:
                df = pd.DataFrame(self.data)
                print("Traffic Analysis (First 5 packets):")
                print(df.head())
            else:
                print("No valid packets captured.")
        except Exception as e:
            print(f"Error during traffic analysis: {e}")

    def save_analysis(self, filename="traffic_analysis.csv"):
        if not self.data:
            print("No data to save.")
            return
        print(f"Saving analysis to {filename}")
        df = pd.DataFrame(self.data)
        df.to_csv(filename, index=False)
        print("File saved successfully.")

In [ ]:
## Run in Jupyter/IPython
interface = input("Enter network interface (e.g., eth0, Ethernet): ")
analyzer = NetworkTrafficAnalyzer(interface)
analyzer.start_capture()
# Use the existing event loop
loop = asyncio.get_event_loop()
loop.run_until_complete(analyzer.analyze_traffic_async(packet_count=50))
analyzer.save_analysis()